Notebook for second problem

In [28]:
ts=zip(range(1,10),range(11,20))

In [31]:
for t in ts:
    print(t)
else:
    print("There was no premature break") #This is interesting. This part of the code executes only when the loop completes 

There was no premature break
